# Bayesian Parameter Estimation 

© 2018 Griffin Chure. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

--- 

In [251]:
import sys
sys.path.insert(0, '../../')
import numpy as np
import pandas as pd
import bokeh.io 
import bokeh.plotting
import pystan
import mut.bayes
import mut.thermo
import tqdm
import bebi103.viz
constants = mut.thermo.load_constants()
bokeh.io.output_notebook()

# For inline stan functions
%load_ext stanmagic

Loading BokehJS ...

The stanmagic extension is already loaded. To reload it, use:
  %reload_ext stanmagic


In this notebook, we cover the practical implementation of the Bayesian parameter estimation used in this work. For pedagogical reasons, we work through estimating the probability distribution for the DNA binding energy of a single DNA binding mutant, Q21M. While the supplemental information covers the models, results, and calibrations for the other mutants and inferential approaches. To begin, we will load the compiled fold-change data and extract the measurements from Q21M with 260 repressors per cell. 

In [ ]:
# Load the data and segregate in to classes
data = pd.read_csv('../../data/csv/compiled_data.csv')
Q21M = data[(data['mutant']=='Q21M') & (data['operator']=='O2') &
           (data['repressors']==260)]

## The Posterior Probability Distribution

$$
\text{fold-change} = \left(1 + {\left(1 + {c \over K_A}^2\right) \over \left(1 + {c \over K_A}^2\right) + e^{-\beta\Delta\varepsilon_{AI}}\left(1 + {c \over K_I}^2\right)}{R \over N_{NS}}e^{-\beta\Delta\varepsilon_{RA}}\right)^{-1}
$$

Under our näive hypothesis, we are assuming that mutations in the DNA binding domain alters *only* the DNA binding energy $\Delta\varepsilon_{RA}$. With a single-parameter model, we can write Bayes' theorem to say that the posterior probability distribution 
of $\Delta\varepsilon_{RA}$ given our data and literature values for the parameters as

$$
g(\Delta\varepsilon_{RA}\,\vert c, R, K_A, K_I, \Delta\varepsilon_{AI}, D) = {f(D \,\vert\, \Delta\varepsilon_{RA}, c, R, K_A, K_I, \Delta\varepsilon_{AI}) g(R, K_A, K_I, \Delta\varepsilon_{AI}, \Delta\varepsilon_{RA}) \over f(D)}
$$

where the notation $g$ and $f$ represent probability densities over parameters and data respectively and $D$ represents the set of all measured data. In this notebook, we take all literature values for the known parameters as $\delta$-functions with zero error. For notational simplicity, we will group all of these known parameters together as 

$$
\theta = \{R, K_A, K_I, N_{NS}, \Delta\varepsilon_{AI}\}.
$$

We are now left to define two elements -- the likelihood $f(D\,\vert\, \theta, \Delta\varepsilon_{RA})$ and the prior $g(\Delta\varepsilon_{RA})$. Our model, presented in Eq. 1, computes the *mean* fold-change in gene expression of a population. Our data consists of a series of measurements of this mean. As each measurement is independent, it is reasonable to assume that each measurement is normally distributed about a true value, $\mu_\text{fc}$. For a set of $N$ measurements, the likelihood can be written as

$$
f(\mathbf{D}\,\vert \mathbf{c}, \theta, \Delta\varepsilon_{RA}, \sigma) = \sum\limits_i^N {1 \over \sqrt{2\pi\sigma^2}}\exp\left[-{\left(D_i - \mu_\text{fc}\right)^2 \over 2\sigma^2}\right].
$$

which is the standard definition of the Gaussian distribution. We can simplify this notationally to 

$$
D_i \sim \text{Norm}(\mu_{fc}, \sigma)\, \forall i.
$$

Note that we have now introduced a new parameter $\sigma$ that will need to be estimated along with the DNA binding energy $\Delta\varepsilon_{RA}$. As such, we must introduce a prior for $\sigma$ along with $\Delta\varepsilon_{RA}$.


## Prior Predictive Checks

First, we'll perform the prior predictive checks for inference of the DNA binding energy of the DNA binding mutants. For this class of mutant, we assume knowledge of the inducer binding constants and repressor copy numbers as $\delta$ functions at the literature values. Using $g$ and $f$ to represent probability densities over parameters and data respectively, we can state in general terms that our posterior probability distribution for $\Delta\varepsilon_{RA}$ is

$$
g(\Delta\varepsilon_{RA}, \sigma \,\vert\, \overbrace{R, K_A, K_I, \Delta\varepsilon_{AI}, c}^\Theta, D) = {f(D\,\vert\, \Theta, \Delta\varepsilon_{RA}, \sigma)g(\Delta\varepsilon_{RA})g(\sigma) \over f(D)}\tag{1},
$$

The likelihood ($f(D\, \vert \Theta, \Delta\varepsilon_{RA}, \sigma))$) describes the likelihood of observing our data $D$ given our parameter values. The product of the likelihood and the priors $g(\Delta\varepsilon_{RA})g(\sigma)$ is often called the *joint distribution* $\pi(D, \Theta, \Delta\varepsilon_{RA}, \sigma)$. Before we consider the data, however, we should ensure that our choice of priors are physically and physiologically relevant. 





In [253]:
# Perform simulations.
n_sim = 500
n_draws = len(Q21M)

# Instantiate a storage data frame
dfs = []
for i in range(n_sim):
    # Draw values from the prior
    epRA_draw = np.random.normal(-10, 10)
    sigma_draw = np.random.gamma(2, 0.01)
    
    # Compute the theoretical fold-change. 
    fc_mu = mut.thermo.SimpleRepression(ep_r=epRA_draw, R=Q21M['repressors'],
                                       ka=constants['Ka'], ki=constants['Ki'],
                                       effector_conc=Q21M['IPTGuM'], 
                                       ep_ai=constants['ep_AI']).fold_change()
    # Draw N samples from the likelihood 
    fc_draws = np.random.normal(fc_mu, sigma_draw)
    
    # Append to the likelihood
    _df = pd.DataFrame(np.array([fc_draws]).T, columns=['fc_draw'])
    _df['ep_RA'] = epRA_draw
    _df['sigma'] = sigma_draw
    _df['sim_idx'] = i + 1 
    _df['IPTGuM'] = Q21M['IPTGuM'].values
    dfs.append(_df)
df = pd.concat(dfs)

With data generated from the prior distributions, we can examine the cumulative distributions of the fold-change to ensure they are physically bounded. 

In [ ]:
p = bokeh.plotting.figure(width=400, height=300, 

                          x_axis_label='fold-change',
                          y_axis_label='cumulative distribution')

for g, d in df.groupby('sim_idx'):
    p.line(np.sort(d['fc_draw']), np.arange(0, len(d), 1) / len(d),
          alpha=0.25, line_width=0.1)
bokeh.io.show(p)

### Simulation Based Calibration

In [245]:
%%stan -v sbc_code
#include ../stan/functions.stan
data {
    // Define the parameters for the simulation based on actual data
    int<lower=0> N; // Number of samples 
    real<lower=0> c[N]; // IPTG concentrations in uM
    real Nns; // Number of nonspecific binding sites
    int<lower=0> n_sites; // Number of allosteric binding sites
    real<lower=0> Ka; // Inducer dissociation constant to active repressor
    real<lower=0> Ki; // Inducer dissociation constant to inactive repressor
    real<lower=0> R;
    real ep_ai; // Energy difference between active and inactive states
    real fc[N]; // Generated ground truth data 
}

transformed data {
    // Log transform the dissocation constants for better sampling
    real ep_a = log(Ka);
    real ep_i = log(Ki);
}

parameters {
    real ep_RA;
    real<lower=0> sigma;
}

model { 
    real mu_sbc[N];
    ep_RA ~ normal(0, 10);
    sigma ~ gamma(2, 10);
    for (i in 1:N) {
        mu_sbc[i] = fold_change(R, Nns, ep_RA, c[i],
                               ep_a, ep_i, ep_ai, n_sites);
        fc[i] ~ normal(mu_sbc[i], sigma);
    }
}


Using pystan.stanc compiler..
-------------------------------------------------------------------------------
Model compiled successfully. Output stored in sbc_code object.
Type sbc_code in a cell to see a nicely formatted code output in a notebook
     ^^^^^^^^
Access model compile output properties
sbc_code.model_file -> Name of stan_file [None]
sbc_code.model_name -> Name of stan model [None]
sbc_code.model_code -> Model code [#include ../stan/fun ....]


In [ ]:
sbc_model = bebi103.stan.StanModel(model_code=sbc_code.model_code)

 With this compiled model, we can iterate through each value of hte prior and perform the inference

In [262]:
sbc_dfs = []
for i in tqdm.tqdm(range(100)):
    d = df[df['sim_idx'] == i]
    
    # Get the ground truth
    epRA_gt = d['ep_RA'].unique()
    sigma_gt = d['sigma'].unique()
    gt = {'ep_RA':epRA_gt,
         'sigma': sigma_gt}
    
    # Generate the data dictionary 
    data_dict = {'N': len(d),
             'c': d['IPTGuM'],
             'n_sites': constants['n_sites'],
             'Nns': constants['Nns'],
             'Ka': constants['Ka'],
             'Ki': constants['Ki'],
             'R': constants['RBS1027'],
             'ep_ai': constants['ep_AI'],
             'fc': d['fc_draw']
            }
    # Sample 
    samples = sbc_model.sampling(data_dict, iter=2000, control=dict(adapt_delta=0.9)).to_dataframe()
    
    
    # Compute the properties for each. 
    _sbc_dfs = pd.DataFrame([])
    for p in ['ep_RA', 'sigma']:
        z_score = (np.mean(samples[p]) - gt[p]) / np.std(samples[p])
        shrinkage = 1 - (np.var(samples[p]) / np.var(df[p].unique()))
        _sbc_dfs['z_score'] = z_score
        _sbc_dfs['shrinkage'] = shrinkage
        _sbc_dfs['param'] = p 
    _sbc_dfs['sim_idx'] = i
    sbc_dfs.append(_sbc_dfs) 
sbc_df = pd.concat(sbc_dfs) 



  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:00<00:40,  2.43it/s]

  2%|▏         | 2/100 [00:01<01:09,  1.41it/s]WARNING:pystan:119 of 4000 iterations ended with a divergence (2.975%).


  3%|▎         | 3/100 [00:04<01:56,  1.20s/it]WARNING:pystan:16 of 4000 iterations ended with a divergence (0.4%).


  4%|▍         | 4/100 [00:06<02:18,  1.44s/it]WARNING:pystan:1 of 4000 iterations ended with a divergence (0.025%).


  5%|▌         | 5/100 [00:08<02:36,  1.65s/it]WARNING:pystan:171 of 4000 iterations ended with a divergence (4.275%).


  6%|▌         | 6/100 [00:10<02:56,  1.88s/it]

  7%|▋         | 7/100 [00:11<02:37,  1.70s/it]

  8%|▊         | 8/100 [00:13<02:22,  1.54s/it]

  9%|▉         | 9/100 [00:14<02:16,  1.50s/it]

 10%|█         | 10/100 [00:15<02:01,  1.35s/it]

 11%|█         | 11/100 [00:17<02:04,  1.39s/it]

 12%|█▏        | 12/100 [00:18<01:51,  1.27s/it]WARNING:pystan:114 of 4000 iterations ended with a divergence (2.85%).


 13%|█▎     

In [268]:
p = bokeh.plotting.figure()
p.circle(sbc_df[sbc_df['param']=='sigma']['shrinkage'], 
        sbc_df[sbc_df['param']=='sigma']['z_score'])
bokeh.io.show(p)

In [267]:
sbc_df[sbc_df['param']=='ep_RA']

,z_score,shrinkage,param,sim_idx


In [234]:
prior_predictive = ppc_model.sampling(data = data_dict,
                                      algorithm='Fixed_param',
                                      warmup=0, chains=1, iter=100)
df_samples = bebi103.stan.extract_array(prior_predictive, name='mu')

bokeh.io.show(bebi103.viz.ecdf_collection(df_samples,
                                         val='fc', 
                                         cats='chain_idx',
                                         alpha=0.1,
                                         show_legend=False))


KeyError: 'divergent__'

In [237]:
prior_predictive.extract()

KeyboardInterrupt: 

In [240]:
prior_sd = bebi103.stan._get_prior_sds(ppc_model, data, ['ep_RA', 'sigma'], 1000)
data_dict = {'N': len(Q21M),
             'c': Q21M['IPTGuM'],
             'n_sites': constants['n_sites'],
             'Nns': constants['Nns'],
             'Ka': constants['Ka'],
             'Ki': constants['Ki'],
             'R': constants['RBS1027'],
             'ep_ai': constants['ep_AI'],
            }

args = (ppc_model, sbc_model,
       data_dict, data_dict,
       ['fc'], ['ep_RA', 'sigma'],
       4, 1000, 2000, 10, None, None,
       prior_sd,
       {'fc': float})

# Assemble the data dictionary and perform the inference. 
res = bebi103.stan._perofm_sbc(args)

ValueError: Variable username is neither int nor float nor list/array thereof

In [243]:
!pip upgrade bebi103


ERROR: unknown command "upgrade"
